In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [5]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = (baseline 
          + trend(time, slope) 
          + seasonality(time, period=365, amplitude=amplitude))
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [6]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [7]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [8]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [9]:
window_size = 30
train_set = windowed_dataset(x_train, 
                             window_size, 
                             batch_size=128, 
                             shuffle_buffer=shuffle_buffer_size)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, 
                           kernel_size=5, 
                           strides=1, 
                           padding='causal', 
                           activation='relu', 
                           input_shape=[None, 1]),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 200)])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 32)          192       
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          16640     
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 64)          24832     
_________________________________________________________________
dense_1 (Dense)              (None, None, 1)           65        
_________________________________________________________________
lambda_1 (Lambda)            (None, None, 1)           0         
Total params: 41,729
Trainable params: 41,729
Non-trainable params: 0
_________________________________________________________________


In [13]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10 ** (epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(
    loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=['mae'])

In [14]:
history = model.fit(train_set, epochs=100, callbacks=[lr_scheduler])

Epoch 1/100
8/8 [==============================] - 9s 1s/step - loss: 69.2930 - mae: 69.8658
Epoch 2/100
8/8 [==============================] - 1s 184ms/step - loss: 68.9513 - mae: 69.3384
Epoch 3/100
8/8 [==============================] - 1s 184ms/step - loss: 68.0121 - mae: 68.4939
Epoch 4/100
8/8 [==============================] - 1s 187ms/step - loss: 66.9382 - mae: 67.4462
Epoch 5/100
8/8 [==============================] - 2s 189ms/step - loss: 65.8310 - mae: 66.2292
Epoch 6/100
8/8 [==============================] - 2s 193ms/step - loss: 64.4242 - mae: 64.8562
Epoch 7/100
8/8 [==============================] - 2s 201ms/step - loss: 62.8836 - mae: 63.3127
Epoch 8/100
8/8 [==============================] - 2s 203ms/step - loss: 61.1172 - mae: 61.6049
Epoch 9/100
8/8 [==============================] - 2s 200ms/step - loss: 59.2147 - mae: 59.7144
Epoch 10/100
8/8 [==============================] - 2s 207ms/step - loss: 57.4881 - mae: 57.6355
Epoch 11/100
8/8 [========================

8/8 [==============================] - 2s 212ms/step - loss: 9.0579 - mae: 9.5738
Epoch 87/100
8/8 [==============================] - 2s 240ms/step - loss: 8.0936 - mae: 8.7694
Epoch 88/100
8/8 [==============================] - 2s 209ms/step - loss: 13.5926 - mae: 14.0098
Epoch 89/100
8/8 [==============================] - 2s 205ms/step - loss: 11.8133 - mae: 12.5379
Epoch 90/100
8/8 [==============================] - 2s 204ms/step - loss: 17.7900 - mae: 18.0478
Epoch 91/100
8/8 [==============================] - 2s 217ms/step - loss: 18.1584 - mae: 18.2710
Epoch 92/100
8/8 [==============================] - 2s 202ms/step - loss: 13.1273 - mae: 13.4447
Epoch 93/100
8/8 [==============================] - 2s 202ms/step - loss: 12.8496 - mae: 13.2617
Epoch 94/100
8/8 [==============================] - 2s 204ms/step - loss: 20.4376 - mae: 20.5985
Epoch 95/100
8/8 [==============================] - 2s 206ms/step - loss: 14.9828 - mae: 15.4311
Epoch 96/100
8/8 [=============================

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
#batch_size = 16
dataset = windowed_dataset(
    x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, 
                         kernel_size=3,
                         strides=1, 
                         padding="causal",
                         activation="relu",
                         input_shape=[None, 1]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 200)])

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(dataset,epochs=500)

In [ ]:
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, rnn_forecast)

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

In [1]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
mae=history.history['mae']
loss=history.history['loss']

epochs=range(len(loss)) # Get number of epochs

#------------------------------------------------
# Plot MAE and Loss
#------------------------------------------------
plt.plot(epochs, mae, 'r')
plt.plot(epochs, loss, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()

epochs_zoom = epochs[200:]
mae_zoom = mae[200:]
loss_zoom = loss[200:]

#------------------------------------------------
# Plot Zoomed MAE and Loss
#------------------------------------------------
plt.plot(epochs_zoom, mae_zoom, 'r')
plt.plot(epochs_zoom, loss_zoom, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

plt.figure()

NameError: name 'history' is not defined